In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import re

from batch_generators.java_batch_generator import JavaBatchGenerator

%load_ext autoreload
%autoreload 2

/home/sven/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data_directory = '/home/sven/java_github_corpus'

In [ ]:
from collections import Counter
import javalang

subdirectories = [name for name in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, name))]
count = len(subdirectories)
counter = Counter()
for i, s in enumerate(subdirectories):
    print('project {}/{}'.format(i, count))
    files = [name for name in os.listdir(os.path.join(data_directory, s)) 
              if not os.path.isdir(os.path.join(data_directory, s, name))]
    for file in files:
        with open(os.path.join(data_directory, s, file), 'r') as file_data:
            tokens = list(javalang.tokenizer.tokenize(file_data.read()))
            counter.update(tokens)

project 0/14435
project 1/14435
project 2/14435
project 3/14435
project 4/14435
project 5/14435
project 6/14435
project 7/14435
project 8/14435
project 9/14435
project 10/14435
project 11/14435
project 12/14435
project 13/14435
project 14/14435
project 15/14435
project 16/14435
project 17/14435
project 18/14435
project 19/14435
project 20/14435
project 21/14435
project 22/14435
project 23/14435
project 24/14435
project 25/14435
project 26/14435
project 27/14435
project 28/14435
project 29/14435
project 30/14435
project 31/14435
project 32/14435
project 33/14435
project 34/14435
project 35/14435
project 36/14435
project 37/14435
project 38/14435
project 39/14435
project 40/14435
project 41/14435
project 42/14435
project 43/14435
project 44/14435
project 45/14435
project 46/14435
project 47/14435
project 48/14435
project 49/14435
project 50/14435
project 51/14435
project 52/14435
project 53/14435
project 54/14435
project 55/14435
project 56/14435
project 57/14435
project 58/14435
project

In [15]:
batch_generator = JavaBatchGenerator(data_directory)
input_iterator = batch_generator.train_batch_generator(8)

In [19]:
y, x, b, z = input_iterator.next()

In [20]:
c = np.packbits(b, axis=2)
for e in c:
    s = ''
    for f in e:
        s += chr(f[0])
    print(s)
print(z)

����������������������������������������������������;)"pleh--  "(nltnirp.tuo.metsyS
���������������������������������������������������������������������������:tluafed
������������������������������������������������������������������;)(nim regetnIgiB
���������������������������������;"EL" = emaNnoitcirtseR gnirtS citats lanif cilbup
���������������������������������������������������;teSrepapllaWsIm naeloob etavirp
�������������������������������������������{ )sgra ][gnirtS(niam diov citats cilbup
����������������������������������������������������������������������������������}
;noitpecxEesraPatadateM sworht )mh >gnirtS ,gnirtS<paMhsaH(etadilav atadateMseliTbM
[31, 8, 17, 50, 32, 40, 1, 83]


In [26]:
a = np.array([2**15], dtype='uint16')
b = np.unpackbits(a.view('uint8'))
print b

[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]


In [11]:
tf.reset_default_graph()
target = tf.constant([[1,2,128,128],[3,4,5,128]], dtype=tf.int32)
pad = tf.constant(128, dtype=tf.int32)
b = tf.map_fn(lambda x: tf.map_fn(lambda y: tf.logical_not(tf.equal(y, pad)), x, dtype=tf.bool), target, dtype= tf.bool)
           
with tf.Session() as sess:
    result = tf.to_float(b).eval()
    print(result)
    print(target.eval())

[[1. 1. 0. 0.]
 [1. 1. 1. 0.]]
[[  1   2 128 128]
 [  3   4   5 128]]


In [60]:
tf.reset_default_graph()
target = tf.constant([[[1],[2],[128],[128]],[[3],[4],[5],[128]]], dtype=tf.int32)
a = tf.reshape(target, [target.shape[0], target.shape[1]])

with tf.Session() as sess:
    print(a.eval())

[[  1   2 128 128]
 [  3   4   5 128]]


In [8]:
tf.reset_default_graph()
batch_size = 128
batch_generator = 'Java' 
data_directory = data_directory
num_layers = 2
num_units = 256

encoder_input = tf.placeholder(tf.int32, shape=(batch_size, None, 1), name='encoder_input')
sequence_lengths = tf.placeholder(tf.int32, shape=(batch_size), name='sequence_lengths')
decoder_input = tf.placeholder(tf.int32, shape=(batch_size, None, 1), name='decoder_input')
target_output = tf.placeholder(tf.int32, shape=(batch_size, None), name='target_output')
target_lengths = tf.placeholder(tf.int32, shape=(batch_size), name="target_lengths")

pad_code = tf.constant(128, dtype = tf.int32)

target_weights = tf.to_int32(tf.map_fn(
                                lambda x: tf.map_fn(
                                            lambda y: tf.logical_not(tf.equal(y, pad_code)), 
                                            x, 
                                            dtype=tf.bool), 
                                target_output, 
                                dtype= tf.bool))

if batch_generator == "Java":
    batch_generator = JavaBatchGenerator(data_directory)
elif batch_generator == "Text":
    raise NotImplementedError("TextBatchGenerator is not implemented yet")
else:
    raise ValueError("batch_generator argument not recognized; must be one of: "
                     "Java, Text")

projection_layer = tf.layers.Dense(256, use_bias = False) # 256 characters can be represented in UTF-8

encoder_layers = [tf.nn.rnn_cell.LSTMCell(num_units) for i in range(num_layers)]
encoder_cell = tf.nn.rnn_cell.MultiRNNCell(encoder_layers)
encoder_outputs, encoder_state = tf.nn.dynamic_rnn(cell = encoder_cell,
                                                    inputs = tf.to_float(encoder_input),
                                                    sequence_length = sequence_lengths,
                                                    dtype = tf.float32)

decoder_layers = [tf.nn.rnn_cell.LSTMCell(num_units) for i in range(num_layers)]
decoder_cell = tf.nn.rnn_cell.MultiRNNCell(decoder_layers)
helper = tf.contrib.seq2seq.TrainingHelper(tf.to_float(decoder_input), target_lengths)
decoder = tf.contrib.seq2seq.BasicDecoder(
    decoder_cell, helper, encoder_state,
    output_layer=projection_layer)
outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder)
logits = outputs.rnn_output

crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels=target_output, logits=logits)
train_loss = (tf.reduce_sum(crossent * tf.to_float(target_weights)) / batch_size)

tf.summary.FileWriter("/home/sven/logs", tf.get_default_graph()).close()

In [35]:
print(ord("\n"))

10
